## Train XGBoost Model using SageMaker Training + Serve (Host) the Model as a SageMaker Endpoint

### 1. Imports 

In [2]:
import pandas as pd
import numpy as np
import json
import os

from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
import sagemaker
import boto3

In [3]:
sagemaker.__version__

'2.107.0'

### 2. Essentials 

In [4]:
# Get the Execution Role for this Notebook and AWS Session to create Clients
sagemaker_execution_role = get_execution_role()
print('Role = {}'.format(sagemaker_execution_role))
session = boto3.Session()

# Clients and Resources
s3 = session.resource('s3')
sagemaker_session = sagemaker.Session()

BUCKET = sagemaker_session.default_bucket()
PREFIX = 'clf'

Role = arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628


##### Upload Train & Test Sets to S3 and Create Pointers to Data

In [5]:
s3.create_bucket(Bucket=BUCKET)

s3.Bucket(name='sagemaker-us-east-1-119174016168')

In [6]:
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'train/train.csv')).upload_file('./DATA/train/train.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'test/test.csv')).upload_file('./DATA/test/test.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'batch_test/batch_test.csv')).upload_file('./DATA/batch_test/batch_test.csv')

##### Create Pointers to the uploaded files 

In [7]:
train_set_location = 's3://{}/{}/train/'.format(BUCKET, PREFIX)
test_set_location = 's3://{}/{}/test/'.format(BUCKET, PREFIX)

In [8]:
print(train_set_location)
print(test_set_location)

s3://sagemaker-us-east-1-119174016168/clf/train/
s3://sagemaker-us-east-1-119174016168/clf/test/


In [9]:
train_set_pointer = TrainingInput(s3_data=train_set_location, content_type='csv')
test_set_pointer = TrainingInput(s3_data=test_set_location, content_type='csv')

In [10]:
print(json.dumps(train_set_pointer.__dict__, indent=2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://sagemaker-us-east-1-119174016168/clf/train/",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


### 3. Train a Model using SageMaker + Builtin XgBoost Algorithm

In [11]:
container_uri = sagemaker.image_uris.retrieve(region=session.region_name, 
                                              framework='xgboost', 
                                              version='1.0-1', 
                                              image_scope='training')

In [12]:
xgb = sagemaker.estimator.Estimator(image_uri=container_uri,
                                    role=sagemaker_execution_role, 
                                    instance_count=2, 
                                    instance_type='ml.m5.large',
                                    output_path='s3://{}/{}/model-artifacts'.format(BUCKET, PREFIX),
                                    sagemaker_session=sagemaker_session,
                                    base_job_name='classifier')

xgb.set_hyperparameters(objective='binary:logistic',
                        num_round=100)

In [ ]:
xgb.fit({'train': train_set_pointer, 'validation': test_set_pointer})

2022-10-04 20:54:21 Starting - Starting the training job...
2022-10-04 20:54:45 Starting - Preparing the instances for trainingProfilerReport-1664916861: InProgress
......
2022-10-04 20:55:45 Downloading - Downloading input data......
2022-10-04 20:56:45 Training - Downloading the training image...
2022-10-04 20:57:20 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[20:57:24] 7461x4 matrix with 29844 entries loaded from /opt/ml/input/data/train?format=csv&l

### 4. Host the Trained Model as a SageMaker Endpoint (using Estimator object)

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=2,
                           instance_type='ml.m5.large')

### 5. Real Time Inference from the Endpoint

In [ ]:
csv_serializer = CSVSerializer()
predictor = Predictor(endpoint_name=xgb_predictor.endpoint_name, 
                      serializer=csv_serializer)

In [ ]:
test_df = pd.read_csv('./DATA/test/test.csv', names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [ ]:
test_df.head()

In [ ]:
X = test_df.sample(1)
X

In [ ]:
X = X.values[0]
X[1:]

In [ ]:
payload = X[1:]

In [ ]:
%%time

predicted_class_prob = predictor.predict(payload).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    print('PREDICTION = NOT DIABETIC')
else:
    print('PREDICTION = DIABETIC')
print()

### 5. Evaluate Hosted Model for Accuracy

In [ ]:
predictions = []
expected = []
correct = 0
for row in test_df.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class_prob = predictor.predict(payload).decode('utf-8')
    predicted_class = 1
    if float(predicted_class_prob) < 0.5:
        predicted_class = 0  
    if predicted_class == expected_class:
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

In [ ]:
print('Accuracy = {:.2f}%'.format(correct/len(predictions) * 100))

#### Confusion Matrix

In [ ]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)